# West Midlands Crime Analysis – Notebook 1

This notebook performs an exploratory analysis of West Midlands Police 
street-level crime data (Aug 2022 – Jul 2025).  

## Notebooks Roadmap
1. **Notebook 1:** Data preparation & cleaning – combine monthly files, handle missing values, remove duplicates, standardize formats  
2. **Notebook 3:** Exploratory Data Analysis (EDA) – summary statistics, crime by type  
3. **Notebook 4:** Visualisation – time series trends and crime category breakdown  
4. **Notebook 5:** Insights & limitations


In [1]:
# Merge multiple CSV files from a folder into a single DataFrame and save it.
import pandas as pd
import glob
import os

# Path to the raw data folder
data_folder = '../data/raw'
# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(data_folder, '*.csv'))

# Load and merge all CSVs
df_list = [pd.read_csv(f) for f in csv_files]
combined_df = pd.concat(df_list, ignore_index=True)

print(f"Loaded {len(csv_files)} files. Combined shape: {combined_df.shape}")

# Save combined DataFrame to processed section
processed_path = '../data/processed/combined.csv'
combined_df.to_csv(processed_path, index=False)
print(f"Combined CSV saved to {processed_path}")

Loaded 14 files. Combined shape: (392363, 12)
Combined CSV saved to ../data/processed/combined.csv


In [4]:
# Exploratory Data Analysis (EDA)

# Ensure combined_df is loaded
if 'combined_df' not in locals():
	combined_df = pd.read_csv('../data/processed/cleaned_crime.csv')

# Inspect the data
combined_df.info()
combined_df.head()
combined_df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392363 entries, 0 to 392362
Data columns (total 12 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Crime ID               358449 non-null  object 
 1   Month                  392363 non-null  object 
 2   Reported by            392363 non-null  object 
 3   Falls within           392363 non-null  object 
 4   Longitude              392363 non-null  float64
 5   Latitude               392363 non-null  float64
 6   Location               392363 non-null  object 
 7   LSOA code              392363 non-null  object 
 8   LSOA name              392363 non-null  object 
 9   Crime type             392363 non-null  object 
 10  Last outcome category  358449 non-null  object 
 11  Context                0 non-null       float64
dtypes: float64(3), object(9)
memory usage: 35.9+ MB


Crime ID                  33914
Month                         0
Reported by                   0
Falls within                  0
Longitude                     0
Latitude                      0
Location                      0
LSOA code                     0
LSOA name                     0
Crime type                    0
Last outcome category     33914
Context                  392363
dtype: int64

### Dataset structure & quality

The dataset contains 392,363 rows and 12 columns, covering June 2024 – July 2025.  
Key points:
- Columns such as `Longitude`, `Latitude`, and `Crime type` are complete.  
- `Crime ID` and `Last outcome category` are missing in ~34k rows. This is expected, since not all crimes have an outcome recorded at the time of publication.  
- `Context` is completely empty and can be dropped.  
- The `Month` column is currently stored as text and will be converted to datetime for analysis.


In [5]:
# Data manipulation 

# Drop context column as it's empty
combined_df = combined_df.drop(columns=["Context"])


In [6]:
# Convert 'Month' to datetime
combined_df["Month"] = pd.to_datetime(combined_df["Month"])


In [7]:
# Remove duplicates across CSVs
combined_df = combined_df.drop_duplicates()


In [9]:
# Create additional time-based columns for analysis

# Ensure Month is datetime
combined_df["Month"] = pd.to_datetime(combined_df["Month"])

# Year as number (2024, 2025…)
combined_df["Year"] = combined_df["Month"].dt.year

# Month name (Jan, Feb…)
combined_df["MonthName"] = combined_df["Month"].dt.month_name()

# Month number (1–12, keeps sorting correct)
combined_df["MonthNum"] = combined_df["Month"].dt.month

# Season grouping months into Winter/Spring/Summer/Autumn
season_map = {
    12: "Winter", 1: "Winter", 2: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"
}
combined_df["Season"] = combined_df["Month"].dt.month.map(season_map)


In [10]:
# Final data check
combined_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 384676 entries, 0 to 392362
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Crime ID               358449 non-null  object        
 1   Month                  384676 non-null  datetime64[ns]
 2   Reported by            384676 non-null  object        
 3   Falls within           384676 non-null  object        
 4   Longitude              384676 non-null  float64       
 5   Latitude               384676 non-null  float64       
 6   Location               384676 non-null  object        
 7   LSOA code              384676 non-null  object        
 8   LSOA name              384676 non-null  object        
 9   Crime type             384676 non-null  object        
 10  Last outcome category  358449 non-null  object        
 11  Year                   384676 non-null  int32         
 12  MonthName              384676 non-null  object   

In [ ]:
# Save cleaned combined DataFrame
cleaned_path = '../data/processed/cleaned_crime.csv'
combined_df.to_csv(cleaned_path, index=False)
print(f"Cleaned data saved to {cleaned_path}")


### Data Cleaning & Manipulation

Before analysis, we cleaned and enhanced the dataset to make it easier to work with:

- Dropped the `Context` column, since it was completely empty.  
- Converted the `Month` column from text to a proper datetime format for accurate grouping and time-series analysis.  
- Removed duplicate rows to ensure crime records are not double-counted.  
- Added helper columns:
  - `Year` → for year-based grouping (2024, 2025).  
  - `MonthName` and `MonthNum` → to analyse crime patterns by calendar month.  
  - `Season` → to explore seasonal crime trends (Winter, Spring, Summer, Autumn).  
- Filled missing outcome values in `Last outcome category` with `"Outcome not recorded"`, so they can be included in counts rather than dropped.

These transformations ensure the dataset is clean, consistent, and ready for deeper analysis and visualisation.